# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    
from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils, ml_methods_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, plot_gpfa_utils, gpfa_helper_class
from neural_data_analysis.neural_analysis_tools.align_trials import time_resolved_regression, time_resolved_gpfa_regression,plot_time_resolved_regression
from neural_data_analysis.neural_analysis_tools.align_trials import align_trial_utils
from decision_making_analysis.event_detection import detect_rsw_and_rcap

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_psth import core_stops_psth, psth_postprocessing, psth_stats, compare_events, dpca_utils
from neural_data_analysis.neural_analysis_tools.glm_tools.glm_fit import general_glm_fit, cv_stop_glm, glm_fit_utils, variance_explained, glm_runner
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_plotting import plot_spikes, plot_glm_fit, plot_tuning_func, compare_glm_fit
from neural_data_analysis.design_kits.design_around_event import event_binning, stop_design, cluster_design, design_checks
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_hyperparams import compare_glm_configs, glm_hyperparams_class
from neural_data_analysis.topic_based_neural_analysis.ff_visibility import ff_vis_epochs, vis_design
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.get_stop_events import get_stops_utils, collect_stop_data, assemble_stop_design
from neural_data_analysis.design_kits.design_by_segment import spike_history

import sys
import math
import gc
import subprocess
from pathlib import Path

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi
import cProfile
import pstats

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr
import statsmodels.api as sm

import json

# Neuroscience specific imports
import neo
import rcca

# To fit gpfa
import numpy as np
from importlib import reload
from scipy.integrate import odeint
import quantities as pq
import neo
from elephant.spike_train_generation import inhomogeneous_poisson_process
from elephant.gpfa import GPFA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from elephant.gpfa import gpfa_core, gpfa_util

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

print("done")


%load_ext autoreload
%autoreload 2
%matplotlib inline

# retrieve data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0413"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0321"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0329"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0403"

In [ ]:
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0312"

# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0316"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0327"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0328"

In [ ]:
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0327"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0301"

pn, datasets, comparisons = collect_stop_data.collect_stop_data_func(
    raw_data_folder_path)

globals().update(datasets)


# FF visible data

Each event corresponds to a visible segment of a firefly and events are ordered by the firefly’s visible onset time. If another firefly becomes visible before the current one stops being visible, the event is reassigned to the newly visible firefly at the moment it appears.

Also note that, the segment duration (a.k.a. for each ff becoming visible) is currently defined as [start_visible_time - pre_s, start_visible_time + post_s]. Right now we use pre_s=0, post_s=1.0.

In [ ]:
pn.make_or_retrieve_ff_dataframe()
df = pn.ff_dataframe.copy()

# minimal: detect runs, no merging (each run is its own cluster)
df2 = ff_vis_epochs.compute_visibility_runs_and_clusters(
    df, ff_col='ff_index', t_col='point_index', time_col='time', vis_col='visible',
    chunk_merge_gap=0.05,    # seconds: merge *raw* runs into chunks if gap <= this
    cluster_merge_gap=1
)

df2 = ff_vis_epochs.add_global_visibility_bursts(df2, global_merge_gap=0.25)
#df2 = ff_vis_epochs.add_global_vis_cluster_id(df2, group_cols=None, nullable_int=True)
df2 = ff_vis_epochs.add_global_vis_chunk_id(df2, group_cols=None, nullable_int=True)
#df2 = ff_vis_epochs.add_global_vis_cluster_id(df2, group_cols=None, nullable_int=True)

vis_df = df2.loc[df2['visible'] == 1].copy()

In [ ]:
# based on any ff visible
sequential_vis_df = vis_df[['ff_index', 'ff_vis_start_time', 'ff_vis_end_time',
                           'global_vis_chunk_id', 'global_burst_id', 'global_burst_start_time','global_burst_end_time',
                           'global_burst_duration','global_burst_size',
                           #'global_burst_prev_start_time','global_burst_prev_end_time'
                           ]].drop_duplicates().reset_index(drop=True)
sequential_vis_df = sequential_vis_df.sort_values('ff_vis_start_time').reset_index(drop=True)
sequential_vis_df['prev_time'] = sequential_vis_df['ff_vis_start_time'].shift(1)
sequential_vis_df['next_time'] = sequential_vis_df['ff_vis_start_time'].shift(-1)


new_seg_info = event_binning.pick_event_window(sequential_vis_df,
                                                event_time_col='ff_vis_start_time',
                                                prev_event_col='prev_time',
                                                next_event_col='next_time',
                                                pre_s=0.1, post_s=0.5, min_pre_bins=2, min_post_bins=3, bin_dt=pn.bin_width)
new_seg_info['event_id'] = new_seg_info['global_vis_chunk_id']
new_seg_info['event_time'] = new_seg_info['ff_vis_start_time']
new_seg_info.iloc[20:35]

In [ ]:
new_seg_info = event_binning.pick_event_window(sequential_vis_df,
                                                event_time_col='ff_vis_start_time',
                                                prev_event_col='prev_time',
                                                next_event_col='next_time',
                                                pre_s=0, post_s=1.0, min_pre_bins=2, min_post_bins=3, bin_dt=pn.bin_width)
new_seg_info['event_id'] = new_seg_info['global_vis_chunk_id']
new_seg_info['event_time'] = new_seg_info['ff_vis_start_time']
new_seg_info.iloc[20:35]

In [ ]:
events_with_stats = sequential_vis_df[['global_vis_chunk_id', 'global_burst_id', 'ff_vis_start_time', 'ff_vis_end_time']].copy()
events_with_stats = sequential_vis_df.rename(columns={'global_vis_chunk_id': 'event_id', 
                                           'global_burst_id': 'event_cluster_id', 
                                           'ff_vis_start_time': 'event_id_start_time', 
                                           'ff_vis_end_time': 'event_id_end_time'})

# Prepare for GLM

## binned_feats

In [ ]:
reload(event_binning)
reload(stop_design)
reload(cluster_design)
reload(design_checks)
reload(general_glm_fit)
reload(glm_fit_utils)
reload(cv_stop_glm)
reload(plot_tuning_func)

In [ ]:

binned_spikes, binned_feats, offset_log, meta_used, meta_groups = assemble_stop_design.build_stop_design(new_seg_info, events_with_stats, 
                                                                             pn.monkey_information, 
                                                                             pn.spikes_df, pn.ff_dataframe, 
                                                                             datasets=datasets,
                                                                             bin_dt=pn.bin_width, 
                                                                             add_ff_visible_info=True,
                                                                             add_retries_info=False)

if 'global_burst_id' not in meta_used.columns:
    meta_used =meta_used.merge(new_seg_info[['event_id', 'global_burst_id']], on='event_id', how='left')
    
binned_feats_sc, scaled_cols = event_binning.selective_zscore(binned_feats)
binned_feats_sc = sm.add_constant(binned_feats_sc, has_constant='add')
print('Scaled columns:', scaled_cols)

df_X = binned_feats_sc.copy()
df_Y = binned_spikes.copy()

In [ ]:
new_seg_info['ok_window'].unique()

In [ ]:
new_seg_info

## check design

In [ ]:
df_X = df_X.drop(columns=['prepost*speed', 'prepost', 'rcos_-0.16s', 'rcos_-0.24s'], errors='ignore')

In [ ]:
# from neural_data_analysis.topic_based_neural_analysis.full_session import selected_raw_data_features, selected_pn_design_features, selected_stop_design_features
# print(f'set(df_X.columns) - set(selected_stop_design_features.full_stop_design_predictors): {set(df_X.columns) - set(selected_stop_design_features.full_stop_design_predictors)}')
# df_X_sub = df_X[selected_stop_design_features.full_stop_design_predictors]

In [ ]:
cols_path = os.path.join(pn.planning_and_neural_folder_path, 'glm_on_ff_vis', 'selected_cols.json')
X_pruned, vif_report = design_checks.load_or_compute_selected_cols(df_X, cols_path, exists_ok=True)

## see features

In [ ]:
df_X.columns.tolist()

## fit

In [ ]:
save_dir = os.path.join(pn.planning_and_neural_folder_path, 'glm_on_ff_vis', 'glm_fit')

report = general_glm_fit.glm_mini_report(
    df_X=df_X, df_Y=df_Y, offset_log=offset_log,
    cov_type='HC1', 
    fast_mle=True,
    do_inference=False, 
    make_plots=True,
    show_plots=True,
    save_dir=save_dir,
)

# Get spike history

In [ ]:
bin_df = spike_history.make_bin_df_from_stop_meta(meta_used)
dt = pn.bin_width
t_max = 0.20

design_w_history, basis, colnames, meta_groups = (
    spike_history.build_design_with_spike_history_from_bins(
        spikes_df=pn.spikes_df,
        bin_df=bin_df,
        X_pruned=X_pruned,
        meta_groups=meta_groups,
        dt=dt,
        t_max=t_max,
    )
)


In [ ]:
# bin_info = meta_used[['event_id', 'k_within_seg']].copy().rename(
#     columns={'event_id': 'new_segment', 'k_within_seg': 'new_bin'})

In [ ]:
# new_seg_info['new_segment'] = new_seg_info['event_id']

In [ ]:
# dt = pn.bin_width
# t_max = 0.20
# spikes_df=pn.spikes_df

# design_w_history, basis, colnames, meta_groups = spike_history.build_spike_history_design(
#     spikes_df=spikes_df,
#     new_seg_info=new_seg_info,
#     bin_info=bin_info,
#     X_pruned=X_pruned,
#     meta_groups=meta_groups,
#     dt=dt,
#     t_max=t_max,
# )


In [ ]:
# dt = pn.bin_width
# t_max = 0.20

# # 1) Rebin with padding
# design_pad, pad_info = spike_history.rebin_spike_data_with_pad(
#     spikes_df=pn.spikes_df,
#     new_seg_info=new_seg_info,
#     bin_width=dt,
#     t_max=t_max,
# )

# # 2) Precompute history for all neurons
# X_hist, basis, colnames = spike_history.compute_spike_history_designs(
#     design_pad,
#     bin_info,
#     n_pad_bins=pad_info['n_pad_bins'],
#     dt=dt,
#     t_max=t_max,
#     n_basis=5,
# )

# # 3) For now, since we include both self-history and cross-history, the design_df is the same for all neurons
# spike_cols = [c for c in design_pad.columns if c.startswith('cluster_')]

# for target_col in spike_cols:
#     cross_neurons = [c for c in spike_cols if c != target_col]

#     design_w_history, meta_groups = spike_history.add_spike_history_to_design(
#         X_pruned,
#         colnames,
#         X_hist,
#         target_col,
#         include_self=True,
#         cross_neurons=cross_neurons,
#         meta_groups=meta_groups,
#     )
#     break

In [ ]:
design_w_history.columns

# GLM

## class

In [ ]:
df_X = binned_feats_sc.copy()
df_Y = binned_spikes.copy()

output_root = os.path.join(pn.planning_and_neural_folder_path, 'glm_on_ff_vis')

pipeline = glm_runner.GLMPipeline(
    spikes_df=pn.spikes_df,
    bin_df=bin_df,
    df_X=df_X,
    df_Y=df_Y,
    meta_groups=meta_groups,
    bin_width=pn.bin_width,
    output_root=output_root,
)

pipeline.run()
pipeline.plot_comparisons()


In [ ]:
stop!

## just behavioral vars

In [ ]:
df_X = binned_feats_sc.copy()
df_Y = binned_spikes.copy()

In [ ]:
save_dir = os.path.join(pn.planning_and_neural_folder_path, 'glm_on_ff_vis', 'glm_fit')

report0 = general_glm_fit.glm_mini_report(
    df_X=X_pruned, df_Y=df_Y, offset_log=offset_log,
    cov_type='HC1', 
    fast_mle=True,
    do_inference=True, 
    make_plots=True,
    show_plots=True,
    save_dir=save_dir,
)

In [ ]:
coefs_df = report0['coefs_df']
coefs_df[(coefs_df['term'] == 'captured') & (coefs_df['sig_FDR'] == True)].sort_values('p', ascending=True)

In [ ]:
coefs_df['refit_on_support'].value_counts()

In [ ]:
coefs_df.info()

## both (behav and spike history)

In [ ]:
cols_path = os.path.join(pn.planning_and_neural_folder_path, 'glm_on_ff_vis', 'selected_cols_w_spike_history.json')
try:
    with open(cols_path, 'r') as f:
        selected_cols_w_history = json.load(f)
    X_pruned1 = design_w_history[selected_cols_w_history].copy()
    print(f'Loaded selected columns from {cols_path}')
except:
    os.makedirs(os.path.dirname(cols_path), exist_ok=True)
    X_pruned1, vif_report = design_checks.check_design(design_w_history)
    with open(cols_path, 'w') as f:
        json.dump(X_pruned1.columns.tolist(), f)
    print(f'Saved selected columns to {cols_path}')

In [ ]:
save_dir = os.path.join(pn.planning_and_neural_folder_path, 'glm_on_ff_vis', 'glm_fit')

report1 = general_glm_fit.glm_mini_report(
    df_X=X_pruned1, df_Y=df_Y, 
    offset_log=offset_log,
    cov_type='HC1', 
    fast_mle=True,
    do_inference=True, 
    make_plots=True,
    show_plots=True,
    meta_groups=meta_groups,
    save_dir=save_dir,
)

## just spike history

In [ ]:
all_history_cols = [c for c in design_w_history.columns if (c.startswith('cluster_') 
                                                            and c not in binned_feats_sc.columns)]
cols_path = os.path.join(pn.planning_and_neural_folder_path, 'glm_on_ff_vis', 'selected_spike_history_cols.json')
try:
    with open(cols_path, 'r') as f:
        selected_history_cols = json.load(f)
    X_pruned2 = design_w_history[selected_history_cols].copy()
    print(f'Loaded selected columns from {cols_path}')
except:
    os.makedirs(os.path.dirname(cols_path), exist_ok=True)
    X_pruned2, vif_report = design_checks.check_design(design_w_history[all_history_cols])
    with open(cols_path, 'w') as f:
        json.dump(X_pruned2.columns.tolist(), f)
    print(f'Saved selected columns to {cols_path}')

In [ ]:
save_dir = os.path.join(pn.planning_and_neural_folder_path, 'glm_on_ff_vis', 'glm_fit')

report2 = general_glm_fit.glm_mini_report(
    df_X=X_pruned2, df_Y=df_Y, 
    offset_log=offset_log,
    cov_type='HC1', 
    fast_mle=True,
    do_inference=True, 
    make_plots=True,
    show_plots=True,
    meta_groups=meta_groups,
    save_dir=save_dir,
)

# Compare deviance explained

## In-sample

In [ ]:
metrics_by_model = {
    'Behavior only': report0['metrics_df'],
    'Behavior + history': report1['metrics_df'],
    'History only': report2['metrics_df'],
}

compare_glm_fit.plot_insample_model_comparison(metrics_by_model)


## CV

In [ ]:
metrics_by_model = {
    'Behavior only': report0['metrics_df'],
    'Behavior + history': report1['metrics_df'],
    'History only': report2['metrics_df'],
}

compare_glm_fit.plot_cv_model_comparison(metrics_by_model)


# Decoding

In [ ]:
from neural_data_analysis.neural_analysis_tools.decoding_tools.general_decoding import standard_decoding

cols_to_decode = ['log1p_num_ff_visible']

groups = meta_used['global_burst_id']

standard_decoding.population_decoding_cv(
    cols_to_decode=cols_to_decode,
    df_X=df_X,
    df_Y=df_Y,
    groups=groups,
    decoders=('logreg', 'lda'),
    n_splits=5
)
